In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "question": "인터스텔라",
        "answer": "🌌🚀⏳",
    },
    {
       "question": "기생충",
        "answer": "🏠💸🔪",
    },
    {
        "question": "컨택트(Arrival)",
        "answer": "👽🛸🔤",
    },
]

film_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

film_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=film_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "영화에 대해 이모지 가지로 표현 할 수 있어."),
        MessagesPlaceholder(variable_name="history"),
        film_prompt,
        ("human", "{question}?"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)
    

In [46]:
invoke_chain("인터스텔라")
invoke_chain("미나리")
invoke_chain("올드보이")
invoke_chain("태극기휘날리며")
invoke_chain("살인의 추억")

🌌🚀⏳The human mentions "Interstellar" and the AI responds with emojis representing space travel and time. The AI then uses emojis to symbolize growth and family.content='🌌🚀⏳'
🌾👨‍👩‍👧‍👦🌱

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-lKxNfSJvszABJaSoaF0A5Mx2 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-lKxNfSJvszABJaSoaF0A5Mx2 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.